In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def fetch_stock_data(ticker, start_date, end_date):
    """Fetch stock data from Yahoo Finance."""
    try:
        data = yf.download(ticker, start=start_date, end=end_date)
        return data
    except Exception as e:
        logging.error(f"Error fetching data for {ticker}: {e}")
        return None

def calculate_indicators(data, threshold):
    """Calculate technical indicators: Bollinger Bands, VWAP, Squeeze, etc."""
    data['MiddleBB'] = data['Close'].rolling(window=20).mean()
    data['UpperBB'] = data['MiddleBB'] + (data['Close'].rolling(window=20).std() * 2)
    data['LowerBB'] = data['MiddleBB'] - (data['Close'].rolling(window=20).std() * 2)
    data['BB_Width'] = (data['UpperBB'] - data['LowerBB']) / data['MiddleBB']
    data['Squeeze'] = (data['BB_Width'] < threshold).astype(int)
    data['VWAP'] = (data['Close'] * data['Volume']).cumsum() / data['Volume'].cumsum()
    data['Low_Lower_Than_Previous'] = (data['Low'] < data['Low'].shift(1)).astype(int)
    data['Close_Above_VWAP'] = (data['Close'] > data['VWAP']).astype(int)
    data['Volume_Increase'] = (data['Volume'] > data['Volume'].rolling(window=5).mean()).astype(int)

    # Calculate moving average for use as a support level
    data['MA'] = data['Close'].rolling(window=50).mean()

    return data

def detect_spring(data):
    """Identify potential Spring patterns with enhanced criteria."""
    data['Spring'] = ((data['Low_Lower_Than_Previous'] == 1) &
                      (data['Close_Above_VWAP'] == 1) &
                      (data['Volume_Increase'] == 1) &
                      (data['BB_Width'] <= 0.05)).astype(int)  # Consider narrow Bollinger Bands
    return data

def confirm_spring(data):
    """Confirm Spring patterns by checking future price movements and rebounds."""
    squeeze_spring = data[(data['Squeeze'] == 1) & (data['Spring'] == 1)].copy()
    squeeze_spring['Days_Between'] = (squeeze_spring.index.to_series().diff().dt.days.fillna(0).astype(int))

    # Initialize 'Confirmed_Spring' with 0
    squeeze_spring['Confirmed_Spring'] = 0

    # Confirm spring by checking for rebounds within the next 5 days
    for i in range(len(squeeze_spring)):
        date = squeeze_spring.index[i]
        next_days = data.loc[date:].head(6)  # Include the day itself and next 5 days
        if len(next_days) > 1:
            for j in range(1, len(next_days)):  # Start from the next day
                # Check if price rebounds above the moving average and exceeds 2% of the MA
                if (next_days['Close'].iloc[j] > squeeze_spring['MA'].iloc[i] * 1.02) and \
                   (next_days['Low'].iloc[j] < squeeze_spring['MA'].iloc[i]):
                    squeeze_spring.at[date, 'Confirmed_Spring'] = 1
                    # Adjust the date to the exact day where the pattern confirms and subtract one day
                    adjusted_date = next_days.index[j] - pd.Timedelta(days=1)
                    squeeze_spring.index = squeeze_spring.index.where(squeeze_spring.index != date, adjusted_date)
                    break
    
    return squeeze_spring[squeeze_spring['Confirmed_Spring'] == 1]

def analyze_stock_data(ticker, start_date, end_date, threshold):
    """Fetch, analyze, and return data for stocks matching the criteria."""
    data = fetch_stock_data(ticker, start_date, end_date)
    if data is None or data.empty:
        return None
    
    data = calculate_indicators(data, threshold)
    data = detect_spring(data)
    confirmed_springs = confirm_spring(data)
    
    return confirmed_springs[['Close', 'Volume', 'Squeeze', 'Spring', 'Days_Between', 'Confirmed_Spring']]

def fetch_russell_2000_tickers(excel_file_path):
    """Fetch Russell 2000 tickers from an Excel file."""
    try:
        df = pd.read_excel(excel_file_path)
        tickers = df['Ticker'].dropna().tolist()
        return tickers
    except Exception as e:
        logging.error(f"Error fetching Russell 2000 tickers: {e}")
        return []

# Define UI elements for user input
start_date_input = widgets.Text(value='2020-01-01', description='Start Date:', disabled=False)
end_date_input = widgets.Text(value='2024-08-16', description='End Date:', disabled=False)
bollinger_slider = widgets.FloatSlider(value=0.05, min=0.00, max=1.00, step=0.01, description='Bollinger Threshold:', disabled=False)
run_analysis_button = widgets.Button(description="Run Analysis")

# Display widgets
display(start_date_input)
display(end_date_input)
display(bollinger_slider)
display(run_analysis_button)

def on_button_click(button):
    """Run analysis for all S&P 500 and Russell 2000 stocks and save results."""
    # Fetch S&P 500 tickers
    sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()
    
    # Fetch Russell 2000 tickers using the updated function
    russell2000_tickers = fetch_russell_2000_tickers(excel_file_path='C:\\Users\\anmol\\Desktop\\russell_2000.xlsx')

    # Combine both lists of tickers
    all_tickers = sp500_tickers + russell2000_tickers
    
    # Get user input values
    start_date = start_date_input.value
    end_date = end_date_input.value
    bollinger_threshold = bollinger_slider.value
    results = {}

    for ticker in all_tickers:
        logging.info(f"Analyzing {ticker}...")
        result = analyze_stock_data(ticker, start_date, end_date, bollinger_threshold)
        if result is not None:
            results[ticker] = result.head(1)  # Keep only the first confirmed occurrence
    
    if results:
        combined_results = pd.concat(results, names=['Ticker', 'Date'])
        combined_results.to_csv('all_stocks_confirmed_spring_filtered.csv')
        logging.info("Results saved to 'all_stocks_confirmed_spring_filtered.csv'.")
    else:
        logging.info("No stocks with confirmed Spring pattern found.")

# Bind button to callback function
run_analysis_button.on_click(on_button_click)


Text(value='2020-01-01', description='Start Date:')

Text(value='2024-08-16', description='End Date:')

FloatSlider(value=0.05, description='Bollinger Threshold:', max=1.0, step=0.01)

Button(description='Run Analysis', style=ButtonStyle())

NameError: name 'fetch_russell_2000_tickers' is not defined